In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [9]:
data = pd.read_csv('/content/drive/MyDrive/modelingPUBG/data/original_data/train_V2.csv')
data = reduce_mem_usage(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Id               object 
 1   groupId          object 
 2   matchId          object 
 3   assists          int8   
 4   boosts           int8   
 5   damageDealt      float16
 6   DBNOs            int8   
 7   headshotKills    int8   
 8   heals            int8   
 9   killPlace        int8   
 10  killPoints       int16  
 11  kills            int8   
 12  killStreaks      int8   
 13  longestKill      float16
 14  matchDuration    int16  
 15  matchType        object 
 16  maxPlace         int8   
 17  numGroups        int8   
 18  rankPoints       int16  
 19  revives          int8   
 20  rideDistance     float16
 21  roadKills        int8   
 22  swimDistance     float16
 23  teamKills        int8   
 24  vehicleDestroys  int8   
 25  walkDistance     float16
 26  weaponsAcquired  int16  
 27  winPoints   

# 1차 Modeling
    - object dtype(4개) + drop할 columns(4개) drop후 rough하게 modeling

In [10]:
## rough하게 object column 모두 제외하고 모델링
## rankpoint, maxPlace, roadKills, matchDuration, teamKills

data = data.drop(columns=['Id', 'groupId', 'matchId', 'matchType', 'rankPoints', 'maxPlace', 'roadKills', 'matchDuration', 'teamKills'])

In [11]:
data.isna().sum() # winPlacePerc 1개 결측치 drop
data = data.dropna()
data.isna().sum()

assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
numGroups          0
revives            0
rideDistance       0
swimDistance       0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

## train, test split

In [12]:
# train, test split
from sklearn.model_selection import train_test_split

X = data.drop(columns='winPlacePerc')
y = data['winPlacePerc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3557572, 19), (889393, 19), (3557572,), (889393,))

## ML model

In [13]:
# ML model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_absolute_error

In [14]:
# LinearRegression

model_lr = LinearRegression().fit(X_train, y_train)
pred = model_lr.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print('LinearRegression MAE : %f' %mae)

LinearRegression MAE : 0.097418


In [15]:
# Lasso

model_ls = Lasso().fit(X_train, y_train)
pred2 = model_ls.predict(X_test)
mae = mean_absolute_error(pred2, y_test)
print('Lasso MAE : %f' %mae)

Lasso MAE : 0.126241


In [16]:
# Ridge

model_rg = Ridge().fit(X_train, y_train)
pred3 = model_rg.predict(X_test)
mae = mean_absolute_error(pred3, y_test)
print('Ridge MAE : %f' %mae)

Ridge MAE : 0.097417


In [17]:
# LGBMRegressor

model_lgb = LGBMRegressor().fit(X_train, y_train)
pred4 = model_lgb.predict(X_test)
mae = mean_absolute_error(pred4, y_test)
print('LGBMRegressor MAE : %f' %mae)

LGBMRegressor MAE : 0.064046


In [18]:
# XGBRegressor

model_xgb = XGBRegressor().fit(X_train, y_train)
pred5 = model_xgb.predict(X_test)
mae = mean_absolute_error(pred5, y_test)
print('XGBRegressor MAE : %f' %mae)

[06:23:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor MAE : 0.071953


In [ ]:
# RandomForestRegressor

model_rfr = RandomForestRegressor().fit(X_train, y_train)
pred6 = model_rfr.predict(X_test)
mae = mean_absolute_error(pred6, y_test)
print('RandomForestRegressor MAE : %f' %mae)

In [ ]:
# SVR

model_svr = SVR().fit(X_train, y_train)
pred7 = model_rfr.predict(X_test)
mae = mean_absolute_error(pred7, y_test)
print('SVR MAE : %f' %mae)